In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

from scipy.optimize import curve_fit


import pandas as pd
from astropy.table import Table, vstack

size_outside_cut = 500
muon_efficiency_cut = 1
min_impact = 2.2199933748101555
max_impact = 9.983608702234397


In [ ]:
def get_data(basedir):
    def _read_files(basedir):
        for index, f in enumerate(os.listdir(basedir)):
            if f.endswith('.fits'):
                tt = Table.read(os.path.join(basedir, f), format='fits')
                if len(tt):
                    yield tt
    return vstack(list(_read_files(basedir)))

def reweight_gaug(data, zenith):
    zenith = np.deg2rad(zenith)
    weights = np.ones(len(data))
    weights = data['mc_energy']**-0.7 * (1/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(zenith)/115) + 0.054/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(zenith)/850))
    return weights / np.sum(weights)

def plot_reweighted_gaug(data, quantity, zenith=0, bins=100, range=(None, None), log=True, label=None, fit=False, fit_p0=None, fit_bounds=(-np.inf, np.inf), is_data = False):
    if is_data:
        weights = np.ones(len(data))/len(data)
    else:
        weights = reweight_gaug(data, zenith)
    counts, bin_edges, patches = plt.hist(data[quantity], bins=bins, histtype='step', lw=2, log=log, label=label, weights=weights)
    y_limits = plt.gca().get_ylim()
    
    if fit:
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        popt, pcov = curve_fit(fit, bin_centers, counts, p0=fit_p0, bounds=fit_bounds)
        plt.plot(bin_centers, fit(bin_centers, *popt), label=f'Fit of {label}\nFit parameters: {popt}')
        plt.ylim(y_limits)
    
    plt.legend()
    plt.xlim(*range)

# Upload datasets

In [ ]:
winter_data_dir = '/Users/vdk/muons2024/data_quality_winter_2024/tables/'
sim_0015_mirror_align_dir = '/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity_0015alignment/tables/'
sim_0015_mirror_align_focus_offset0_dir = '/Users/vdk/muons2024/psf_work/focus_offset_0/tables/'
sim_00046_mirror_aling_dir = '/Users/vdk/muons2024/lapalma_simulations/proper_mc_config/nsbtune2024year_tuned_reflectivity/tables/'

winter_data = get_data(winter_data_dir)
sim_0015_mirror_align = get_data(sim_0015_mirror_align_dir)
sim_00046_mirror_align = get_data(sim_00046_mirror_aling_dir)
sim_0015_mirror_align_focus_offset0 = get_data(sim_0015_mirror_align_focus_offset0_dir)

good_ring_mask_sim_0015 = sim_0015_mirror_align['good_ring']>0
good_ring_mask_sim_00046 = sim_00046_mirror_align['good_ring']>0
good_ring_mask_sim_0015_focus_offset0 = sim_0015_mirror_align_focus_offset0['good_ring']>0
good_ring_mask_winter = winter_data['good_ring']>0

# Applying cuts

In [ ]:
ring_completeness_cut = 0.9
ring_containment_cut = 0
min_radius_cut = 1
max_radius_cut = 1.3
ring_center_distance_cut = 0.7
impact_distance_cut = 7.5


def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

def apply_cuts(df, completeness_cut, containment_cut, min_radius_cut, max_radius_cut, center_distance_cut, impact_distance_cut):
    return df[
        (df['ring_completeness'] > completeness_cut) &
        (df['ring_containment'] > containment_cut) &
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut) &
        (df['ring_center_x'].abs() < center_distance_cut) &
        (df['ring_center_y'].abs() < center_distance_cut) &
        (df['impact_parameter'] < impact_distance_cut)
    ]

winter_data_cut = apply_cuts(winter_data.to_pandas(), ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for winter_data_cut: {len(winter_data_cut)} ({calculate_survival_percentage(winter_data.to_pandas(), winter_data_cut):.2f}%)")

sim_0015_mirror_align_cut = apply_cuts(sim_0015_mirror_align.to_pandas(), ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for sim_0015_mirror_align_cut: {len(sim_0015_mirror_align_cut)} ({calculate_survival_percentage(sim_0015_mirror_align.to_pandas(), sim_0015_mirror_align_cut):.2f}%)")

sim_00046_mirror_align_cut = apply_cuts(sim_00046_mirror_align.to_pandas(), ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for sim_00046_mirror_align_cut: {len(sim_00046_mirror_align_cut)} ({calculate_survival_percentage(sim_00046_mirror_align.to_pandas(), sim_00046_mirror_align_cut):.2f}%)")

sim_0015_mirror_align_focus_offset0_cut = apply_cuts(sim_0015_mirror_align_focus_offset0.to_pandas(), ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for sim_0015_mirror_align_focus_offset0_cut: {len(sim_0015_mirror_align_focus_offset0_cut)} ({calculate_survival_percentage(sim_0015_mirror_align_focus_offset0.to_pandas(), sim_0015_mirror_align_focus_offset0_cut):.2f}%)")

In [ ]:
plt.figure(figsize=(12, 8))

plot_reweighted_gaug(winter_data_cut, 'ring_radius', zenith=10, is_data=True, label = 'Winter 2024 Good Quality Data')
plot_reweighted_gaug(sim_00046_mirror_align_cut, 'ring_radius', zenith=10, is_data=False, label = 'Sim with 0.0046 deg random alignment (reference)')
plot_reweighted_gaug(sim_0015_mirror_align_cut, 'ring_radius', zenith=10, is_data=False, label = 'Sim with 0.015 deg random alignment')
plot_reweighted_gaug(sim_0015_mirror_align_focus_offset0_cut, 'ring_radius', zenith=10, is_data=False, label = 'Sim with 0.015 deg random alignment and focus offset=0')
plt.xlabel('Ring radius')
plt.ylabel('Normalized counts')

In [ ]:
gaug_weights_0015 = reweight_gaug(sim_0015_mirror_align, 10)
gaug_weights_00046 = reweight_gaug(sim_00046_mirror_align, 10)
data_weights = np.ones(len(winter_data))/len(winter_data)
sim_0015_standard_weights = sim_0015_mirror_align['mc_energy'][good_ring_mask_sim_0015]/np.sum(sim_0015_mirror_align['mc_energy'][good_ring_mask_sim_0015])
sim_00046_standard_weights = sim_00046_mirror_align['mc_energy'][good_ring_mask_sim_00046]/np.sum(sim_00046_mirror_align['mc_energy'][good_ring_mask_sim_00046])

In [ ]:
gaug_weights_0015 = reweight_gaug(sim_0015_mirror_align_cut, 10)
gaug_weights_00046 = reweight_gaug(sim_00046_mirror_align_cut, 10)
data_weights = np.ones(len(winter_data_cut))/len(winter_data_cut)

In [ ]:
name_of_columns = ['ring_size', 'ring_center_x', 'ring_center_y', 'ring_radius', 'num_pixels_in_ring', 'mean_pixel_charge_around_ring']
def plot_columns(df_data, df_sim, columns):
    fig, axes = plt.subplots(3, 2, figsize=(15, 20))
    axes = axes.flatten()

    for i, col in enumerate(columns):
        data_values = df_data[col].dropna()
        sim_values = df_sim[col].dropna()

        combined_values = pd.concat([data_values, sim_values])

        min_value = combined_values.min()
        max_value = combined_values.max()

        total_events = len(combined_values)
        num_bins = int(np.sqrt(total_events))  

        num_bins = max(10, min(num_bins, 100)) 

        bins = np.linspace(min_value, max_value, num_bins + 1)

        axes[i].hist(data_values, weights = data_weights, bins=bins, alpha=0.7, color='k', density=True, label='Data')
        axes[i].hist(sim_values, weights = gaug_weights_00046, bins=bins, alpha=0.7, color='orange', density=True, label='Simulation')

        axes[i].set_title(f'Histogram of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Normalized Frequency')
        axes[i].legend()

    plt.tight_layout()
    #plt.savefig('/Users/vdk/winter_data_vs_sim_0015.png')
    plt.show()
    
plot_columns(df_data=winter_data_cut, df_sim=sim_00046_mirror_align_cut, columns=name_of_columns)

In [ ]:
QUANTILE_CUT = 0.2
def apply_quantile_cut(df, column, quantile):
    quantile_value = df[column].quantile(quantile)
    return df[df[column] < quantile_value]

winter_data_cut_quantile = apply_quantile_cut(winter_data_cut, 'ring_width', QUANTILE_CUT)
sim_00046_mirror_align_cut_quantile = apply_quantile_cut(sim_00046_mirror_align_cut, 'ring_width', QUANTILE_CUT)
sim_0015_mirror_align_cut_quantile = apply_quantile_cut(sim_0015_mirror_align_cut, 'ring_width', QUANTILE_CUT)

plt.figure(figsize=(12, 8))

plt.scatter(winter_data_cut_quantile['ring_radius'], 
            winter_data_cut_quantile['ring_width']/winter_data_cut_quantile['ring_radius'],
            s = 1, 
            label = 'Data')
plt.scatter(sim_00046_mirror_align_cut_quantile['ring_radius'], 
            sim_00046_mirror_align_cut_quantile['ring_width']/sim_00046_mirror_align_cut_quantile['ring_radius'], 
            s=1, alpha=0.5, label='Sim with 0.0046 deg random alignment')

plt.scatter(sim_0015_mirror_align_cut_quantile['ring_radius'], 
            sim_0015_mirror_align_cut_quantile['ring_width']/sim_0015_mirror_align_cut_quantile['ring_radius'], 
            s=1, alpha=0.2, label='Sim with 0.015 deg random alignment')

plt.ylim(0.02, 0.06)
plt.legend()

In [ ]:
winter_data_cut_quantile['radius_bin'] = pd.cut(
    winter_data_cut_quantile['ring_radius'],
    bins=10
)
mean_ring_width_per_bin = winter_data_cut_quantile.groupby('radius_bin')['ring_width'].mean()
mean_ring_width_per_bin = mean_ring_width_per_bin.reset_index()

mean_ring_width_per_bin['bin_mid'] = mean_ring_width_per_bin['radius_bin'].apply(lambda x: x.mid)
# Plotting the data
plt.figure(figsize=(10, 6))
plt.plot(mean_ring_width_per_bin['bin_mid'], mean_ring_width_per_bin['ring_width'], marker='o', linestyle='-')
plt.xlabel('Ring Radius')
plt.ylabel('Mean Ring Width')
plt.title('Mean Ring Width vs. Ring Radius')
plt.grid(alpha=0.5)
plt.ylim(0.02, 0.06)
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Assuming the dataframes are already loaded
# Optional: Drop NaN values
winter_data_cut_quantile = winter_data_cut_quantile.dropna(subset=['ring_radius', 'ring_width'])
sim_00046_mirror_align_cut_quantile = sim_00046_mirror_align_cut_quantile.dropna(subset=['ring_radius', 'ring_width'])
sim_0015_mirror_align_cut_quantile = sim_0015_mirror_align_cut_quantile.dropna(subset=['ring_radius', 'ring_width'])

# Combine 'ring_radius' to get common bin edges
all_radii = pd.concat([
    winter_data_cut_quantile['ring_radius'],
    sim_00046_mirror_align_cut_quantile['ring_radius'],
    sim_0015_mirror_align_cut_quantile['ring_radius']
])

# Define the number of bins
num_bins = 10

# Calculate common bin edges
bin_edges = np.linspace(all_radii.min(), all_radii.max(), num_bins + 1)

# Function to bin data and compute mean
def bin_and_compute_mean(df, bin_edges):
    df['radius_bin'] = pd.cut(df['ring_radius'], bins=bin_edges, include_lowest=True)
    mean_ring_width_per_bin = df.groupby('radius_bin')['ring_width'].mean().reset_index()
    mean_ring_width_per_bin['bin_mid'] = mean_ring_width_per_bin['radius_bin'].apply(lambda x: x.mid)
    return mean_ring_width_per_bin

# Process each dataframe
mean_ring_width_winter = bin_and_compute_mean(winter_data_cut_quantile.copy(), bin_edges)
mean_ring_width_sim_00046 = bin_and_compute_mean(sim_00046_mirror_align_cut_quantile.copy(), bin_edges)
mean_ring_width_sim_0015 = bin_and_compute_mean(sim_0015_mirror_align_cut_quantile.copy(), bin_edges)

# Plotting the data
plt.figure(figsize=(10, 6))

plt.plot(mean_ring_width_winter['bin_mid'], mean_ring_width_winter['ring_width'],
         marker='o', linestyle='-', label='Winter Data')

plt.plot(mean_ring_width_sim_00046['bin_mid'], mean_ring_width_sim_00046['ring_width'],
         marker='s', linestyle='--', label='Sim 00046 Mirror Align')

plt.plot(mean_ring_width_sim_0015['bin_mid'], mean_ring_width_sim_0015['ring_width'],
         marker='^', linestyle='-.', label='Sim 0015 Mirror Align')

plt.xlabel('Ring Radius')
plt.ylabel('Mean Ring Width')
plt.title('Mean Ring Width vs. Ring Radius')
plt.legend()
plt.grid(True)
plt.ylim(0.02, 0.06)
plt.show()